In [10]:
#Random Forest
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
import mltools.dtree as dt;
import sys
sys.path.append('.')
reload(dt);
X = np.genfromtxt("data/X_train.txt",delimiter = None)
Y = np.genfromtxt("data/Y_train.txt",delimiter = None)
Xte = np.genfromtxt("data/X_test.txt",delimiter = None)
rescale = ml.transforms.rescale
Xt,Xv,Yt,Yv = ml.splitData(X,Y,0.80)
Xt, args=ml.transforms.rescale(Xt)
Xte,_=ml.transforms.rescale(Xte,args)
Xv,_=ml.transforms.rescale(Xv,args)
M = Xt.shape[0]
Mv= Xv.shape[0]
rforest = [None]*25
YtHat = np.zeros((M,25))
YvHat = np.zeros((Mv,25))

class randomForest2(ml.base.classifier):
    def __init__(self, learners):
        self.learners=learners
        self.classes=learners[0].classes
    def predictSoft(self,X):
        ysoft = np.zeros((X.shape[0],len(self.classes)));
        for i in range(len(self.learners)): ysoft+=self.learners[i].predictSoft(X);
        return ysoft/len(self.learners);

for l in range(25):
    Xi,Yi = ml.bootstrapData(Xt,Yt, M) # draw this member's random sample of data
    rforest[l] = dt.treeClassify() # and train the model on that draw
    rforest[l].train(Xi,Yi,maxDepth=25,nFeatures=3)
    YtHat[:,l] = rforest[l].predict(Xt) # predict on training data
    YvHat[:,l] = rforest[l].predict(Xv) # and validation data & save results




rf2 = randomForest2(rforest);
print "AUC Train: ",rf2.auc(Xt,Yt)," Valid: ",rf2.auc(Xv,Yv)


Ypred = rf2.predictSoft(Xte)   # make "soft" predictions from your learner  (Mx2 numpy array)
np.savetxt('Yhat.txt', np.vstack( (np.arange(len(Ypred)) , Ypred[:,1]) ).T, 
           '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');
#print("Finished Random Forest with nFeatures ="+str(i))
print("Finished Random Forest")

KeyboardInterrupt: 

In [5]:
#KNN
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
import mltools.dtree as dt;
import sys
sys.path.append('.')
Xtr = X[:10000]
Ytr = Y[:10000]

Xva = X[10000:20000]
Yva = Y[10000:20000]

fs = [
[0, 6, 13]
]

class Ensemble:
    def __init__(self, learners, setfeatures = None):
        self.ensemble = learners
        self.features = setfeatures
        self.classes = learners[0].classes

    def predictSoft(self, X):
        Ypreds = []
        for i, l in enumerate(self.ensemble):
            Xf = X[:,self.features[i]]
            Ypreds.append(l.predictSoft(Xf))
            print("Features finished: " + str(self.features[i]))
        return np.mean(Ypreds, axis = 0)


ensemble = []

for f in fs:
    XtrF = X[:,f]
    l = ml.knn.knnClassify(X=XtrF, Y=Y, K=8)
    ensemble.append(l)

    print("Trained finished: " + str(f))

e = Ensemble(ensemble, fs)

Ypred = e.predictSoft(Xte)
np.savetxt('Yhat.txt', np.vstack( (np.arange(len(Ypred)), Ypred[:,1])).T, '%d, %.2f', header='ID,Prob1', comments='')
print("Finished KNN")

Trained finished: [0]
Trained finished: [6]
Trained finished: [13]
Trained finished: [0, 6, 13]
Features finished: [0]
Features finished: [6]
Features finished: [13]
Features finished: [0, 6, 13]
Finished KNN


In [14]:
#Clustering
import numpy as np
np.random.seed(0)
import mltools as ml
import matplotlib.pyplot as plt   # use matplotlib for plotting with inline plots
%matplotlib inline

X = np.genfromtxt("data/X_train.txt",delimiter=' ')[:,0:2]
#X = X[:,0:2]
Y = np.genfromtxt("data/Y_train.txt",delimiter=' ')
Xt,Xv,Yt,Yv = ml.splitData(X,Y,0.80)

Xe = np.genfromtxt('data/X_test.txt',delimiter=' ')[:,0:2]



def auc(soft,Y):
    """Manual AUC function for applying to soft prediction vectors"""
    indices = np.argsort(soft)         # sort data by score value
    Y = Y[indices]
    sorted_soft = soft[indices]
    
    # compute rank (averaged for ties) of sorted data
    dif = np.hstack( ([True],np.diff(sorted_soft)!=0,[True]) )
    r1  = np.argwhere(dif).flatten()
    r2  = r1[0:-1] + 0.5*(r1[1:]-r1[0:-1]) + 0.5
    rnk = r2[np.cumsum(dif[:-1])-1]
    
    # number of true negatives and positives
    n0,n1 = sum(Y == 0), sum(Y == 1)
    
    # compute AUC using Mann-Whitney U statistic
    result = (np.sum(rnk[Y == 1]) - n1 * (n1 + 1.0) / 2.0) / n1 / n0
    return result




nClust = 16
np.random.seed(0)
Z,T,pZ,ll = ml.cluster.gmmEM(Xt[:10000,:],K=nClust,init='k++',max_iter=50)

Cluster_GMM = ml.bayes.gaussClassify()
# Manually copy the EM Gaussian components into the Gaussian Bayes Classifier:
Cluster_GMM.classes = np.arange(nClust)
Cluster_GMM.means = T['mu']
Cluster_GMM.covars = [ T['sig'][:,:,i]+.05*np.eye(Xt.shape[1]) for i in range(nClust) ]
Cluster_GMM.probs = T['pi']

# Find cluster membership probabilities for each data set:
XtC = Cluster_GMM.predictSoft(Xt)
XvC = Cluster_GMM.predictSoft(Xv)
XeC = Cluster_GMM.predictSoft(Xe)

# Create extended feature set:  features X times membership probability for each cluster
XtC2 = np.einsum('ij,ik->ijk',XtC,Xt).reshape((Xt.shape[0],Xt.shape[1]*nClust))
XvC2 = np.einsum('ij,ik->ijk',XvC,Xv).reshape((Xv.shape[0],Xv.shape[1]*nClust))
XeC2 = np.einsum('ij,ik->ijk',XeC,Xe).reshape((Xe.shape[0],Xe.shape[1]*nClust))

# Regress (should really use a classifier...)
linr2 = ml.linear.linearRegress(XtC2,Yt, reg=1e-3)
Pt3 = linr2.predict(XtC2)[:,0]
Pv3 = linr2.predict(XvC2)[:,0]
Pe3 = linr2.predict(XeC2)[:,0]

#toKaggle('Pe3.csv',Pe3)
#print(Pe3)
#print "3: Clustered LinRegress: MSE ~",linr2.mse(XvC2,Yv),'; AUC = ',auc(Pv3,Yv)
print "AUC Train: ",auc(Pt3,Yt)," Valid: ",auc(Pv3,Yv)
np.savetxt('Yhat.txt', np.vstack(( np.arange(len(Pe3)), Pe3[:])).T, '%d, %.2f', header='ID,Prob1', comments='')
print("Finished Clustering")

AUC Train:  0.6459808872  Valid:  0.648453134619
Finished Clustering
